# Классификация текстов: спам-фильтр для SMS

В этом задании мы взяли открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построили на нем классификатор текстов на эти два класса, оценили его качество с помощью кросс-валидации, протестировали его работу на отдельных примерах, и посмотрели, что будет происходить с качеством, если менять параметры модели.

1. Загрузили датасет. Описание датасета можно посмотреть здесь: https://www.kaggle.com/uciml/sms-spam-collection-dataset.

2. Считайли датасет в Python, выяснили, что используется в качестве разделителей и как проставляются метки классов.

3. Подготовили для дальнейшей работы два списка: список текстов и список соответствующих им меток классов. В качестве метки класса использовали 1 для спама и 0 для "не спама".

In [1]:
import re

file = open('SMSSpamCollection.txt', "r", encoding="utf-8").read()
ham = re.findall('ham\t(.+)\n', file)
spam = re.findall('spam\t(.+)\n', file)

data = ham + spam
labels = [0]*len(ham) + [1]*len(spam)

print('Размер выборки:', len(data))
print('Количество не спама:', len(ham))
print('Количество спама:', len(spam))

Размер выборки: 5573
Количество не спама: 4826
Количество спама: 747


4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получили из текстов матрицу признаков X.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)

5. Сделали классификацию текстов с помощью LogisticRegression() с параметрами по умолчанию и оценили качество sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установили random_state=2. Параметр cv равее 10. В качестве метрики качества использовали f1-меру.

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

random_state = 2
logreg = LogisticRegression(solver='lbfgs')
print(cross_val_score(logreg, X, labels, scoring='f1', cv=10).mean())

0.9311269283144492


6. Далее обучили классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [4]:
from sklearn.pipeline import Pipeline

clf_pipeline = Pipeline(
            [("vectorizer", vectorizer), ("classifier", logreg)])

clf_pipeline.fit(data, labels)

test = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
        "FreeMsg: Txt: claim your reward of 3 hours talk time",
        "Have you visited the last lecture on physics?",
        "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
        "Only 99$"]

print(clf_pipeline.predict(test))

[1 1 0 0 0]


7. Задали в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерили получившееся в кросс-валидации значение f1-меры. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратим внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [5]:
for var in [(2, 2), (3, 3), (1, 3)]:
    vectorizer = CountVectorizer(ngram_range=var)
    X = vectorizer.fit_transform(data)
    random_state = 2
    score = cross_val_score(logreg, X, labels, scoring='f1', cv=10).mean()
    print(var, score)

(2, 2) 0.816782323945987
(3, 3) 0.7250161555467377
(1, 3) 0.922300452240204


8. Повторили аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратим внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, мы отдельно посчитали частоты слов и обучили классификатор.

In [6]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
for var in [(2, 2), (3, 3), (1, 3)]:
    vectorizer = CountVectorizer(ngram_range=var)
    X = vectorizer.fit_transform(data)
    random_state = 2
    score = cross_val_score(nb, X, labels, scoring='f1', cv=10).mean()
    print(var, score)

(2, 2) 0.6454554013558982
(3, 3) 0.37862343087618666
(1, 3) 0.8879054608894993


9. Попробовали использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах.
Качество на кросс-валидации понизилось по сравнению с CountVectorizer на униграммах.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

for vectorizer in [TfidfVectorizer(ngram_range=(1, 1)), CountVectorizer(ngram_range=(1, 1))]:
    X = vectorizer.fit_transform(data)
    random_state = 2
    score = cross_val_score(logreg, X, labels, scoring='f1', cv=10).mean()
    print(vectorizer)
    print(score)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
0.8511210908899957
CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
0.9311269283144492
